<a href="https://colab.research.google.com/github/zhangwenhe1007/CPSC483/blob/main/colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Outline



- Basic operation
- Models
- Datasets
- Training and testing

## Basic operation

In [1]:
# import the pytorch library into environment and check its version
import torch
print("Using torch", torch.__version__)

# check if GPU is available and detectable. cpu is ok for this homework.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using torch 2.5.1+cu124
Using device: cpu


### Create Tensor

Tensor are the central data abstraction in PyTorch. You can generalize it to the concept you already know. For example, a vector is a 1-D tensor, and a matrix is a 2-D tensor. We can easily create a tensor of various shapes and number of dimensions by PyTorch.

In [2]:
# construct a 1-D tensor from a list
x = torch.tensor([1, 2, 3])
print(x)

# construct a 2-D tensor from a list
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(x)

tensor([1, 2, 3])
tensor([[1, 2, 3],
        [4, 5, 6]])


Given a tensor, we can obtain its shape by using `.size` or `.shape` method:

In [4]:
print("Shape of x is", x.shape)

print("Size of x is", x.size())

Shape of x is torch.Size([2, 3, 4])
Size of x is torch.Size([2, 3, 4])


Here are alternatives to create a tensor:

* `torch.zeros`: Creates a tensor filled with zeros
* `torch.ones`: Creates a tensor filled with ones
* `torch.rand`: Creates a tensor with random values uniformly sampled between 0 and 1
* `torch.randn`: Creates a tensor with random values sampled from a normal distribution with mean 0 and variance 1
* `torch.arange`: Creates a 1-D tensor containing the values $N,N+1,N+2,...,M$



In [3]:
# construct a tensor with random numbers with shape (4, 5)
x = torch.rand(4, 5)
print("Tensor created by torch.rand function", x)

# construct a tensor filled with the scalar value 0 with shape (2, 3, 4)
x = torch.zeros(2, 3, 4)
print("Tensor created by torch.arange function", x)

Tensor created by torch.rand function tensor([[0.5598, 0.8143, 0.1379, 0.4246, 0.1844],
        [0.4747, 0.7054, 0.1952, 0.5594, 0.2292],
        [0.5106, 0.5162, 0.9505, 0.2154, 0.4511],
        [0.6094, 0.2101, 0.6925, 0.9734, 0.5758]])
Tensor created by torch.arange function tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


### Tensor Operations

PyTorch supports various tensor operations. A full list of operations can be found in the [PyTorch documentation](https://pytorch.org/docs/stable/tensors.html#). For example, we can add two tensors with same shape:

In [5]:
x1 = torch.rand(2, 2)
x2 = torch.rand(2, 2)
x3 = x1 + x2  # return the sum of x1 and x2

print("x1:", x1)
print("x2:", x2)
print("x3:", x3)

x1: tensor([[0.6367, 0.6002],
        [0.8809, 0.6014]])
x2: tensor([[0.4451, 0.4498],
        [0.6449, 0.0420]])
x3: tensor([[1.0818, 1.0500],
        [1.5258, 0.6434]])


We also can calculate the matrix product between two tensors:

In [6]:
x = torch.rand(3)
W = torch.rand(2, 3)
h = torch.matmul(W, x)  # return the product between W and x

print("shape of x:", x.shape, "x:", x)  # shape: [3]
print("shape of W:", W.shape, "W:", W)  # shape: [2, 3]
print("shape of h:", h.shape, "h:", h)  # shape: [2]

shape of x: torch.Size([3]) x: tensor([0.4458, 0.2247, 0.9680])
shape of W: torch.Size([2, 3]) W: tensor([[0.2749, 0.2249, 0.9730],
        [0.2304, 0.3719, 0.3922]])
shape of h: torch.Size([2]) h: tensor([1.1150, 0.5659])


Another common operation aims at changing the shape of a tensor. A tensor'size can be re-organized to any other shape with the same number of elements. In PyTorch, this operation is called `view` or `reshape`:

In [7]:
x = torch.rand(2, 3)
x1 = x.view(6)
x2 = x.reshape(3, 2)

print("shape of x:", x.shape, "x:", x)  # shape: [2, 3]
print("shape of x1:", x1.shape, "x1:", x1)  # shape: [6]
print("shape of x2:", x2.shape, "x2:", x2)  # shape: [3, 2]

shape of x: torch.Size([2, 3]) x: tensor([[0.7446, 0.8502, 0.7441],
        [0.8195, 0.3318, 0.7086]])
shape of x1: torch.Size([6]) x1: tensor([0.7446, 0.8502, 0.7441, 0.8195, 0.3318, 0.7086])
shape of x2: torch.Size([3, 2]) x2: tensor([[0.7446, 0.8502],
        [0.7441, 0.8195],
        [0.3318, 0.7086]])


### Question 1 (5 points)

Given a 1-D tensor, what is the index of its maximum value?


In [8]:
def get_idx_max(x):
  # TODO: Implement a function that takes a tensor object
  # and returns the index of its maximum value.

  max_idx = -1

  ############# Your code here ############
  ## (~1 line of code)
  max_idx = torch.argmax(x)

  #########################################

  return max_idx

x = torch.rand(5)
max_idx = get_idx_max(x)
print('index of the maximum value of {} is {}'.format(x, max_idx))

index of the maximum value of tensor([0.5360, 0.3206, 0.1231, 0.9899, 0.6427]) is 3


## Models

We can use PyTorch to build deep learning model. Here we will give an example of using multi-layer perceptron (MLP) to perform image classification. We'll start with building a MLP.


In [10]:
# import neural network module of PyTorch
import torch.nn as nn
import torch.nn.functional as F

A MLP is built by stacking multiple linear layers:

$$Linear(X):=A X + b$$

with $X\in \mathbb{R}^{n\times k}$, $A\in \mathbb{R}^{m\times n}$, $b\in\mathbb{R}^{m\times 1}$. We can implement a linear layer with `nn.Linear`:



In [11]:
linear = nn.Linear(5, 10)  # creat a linear layer with n=5, m=10

print(linear.weight.shape)  # [10, 5]
print(linear.bias.shape)  # [10]

torch.Size([10, 5])
torch.Size([10])


We can feed a tensor into the linear module to perform linear transformation:

In [12]:
x = torch.rand(6, 5)

print("x:", x)
print("Linear(x):", linear(x))

x: tensor([[0.7047, 0.7927, 0.0912, 0.3590, 0.2222],
        [0.5396, 0.7219, 0.0596, 0.3936, 0.8781],
        [0.3786, 0.5201, 0.0064, 0.6265, 0.9102],
        [0.9125, 0.0757, 0.0500, 0.6784, 0.8381],
        [0.9900, 0.6921, 0.3457, 0.3020, 0.9030],
        [0.5625, 0.4057, 0.4559, 0.4174, 0.4407]])
Linear(x): tensor([[-0.2659, -0.7760, -0.0179, -0.6385, -0.5730, -0.6446, -0.3341, -0.2449,
         -0.1309, -0.0188],
        [-0.0602, -1.0056, -0.2735, -0.7382, -0.7438, -0.8656, -0.1910, -0.4006,
         -0.3486,  0.1490],
        [-0.0600, -1.0095, -0.3019, -0.7237, -0.6759, -0.7469, -0.2388, -0.3694,
         -0.3671,  0.2730],
        [-0.0816, -1.1028, -0.0805, -0.7027, -0.4239, -0.8587,  0.1041, -0.1865,
         -0.6356,  0.3444],
        [-0.0035, -1.0945, -0.1620, -0.7885, -0.6264, -1.0300,  0.0503, -0.3015,
         -0.4632,  0.0679],
        [-0.0855, -0.7327, -0.0119, -0.5119, -0.4131, -0.6049, -0.1476, -0.1424,
         -0.2686,  0.0753]], grad_fn=<AddmmBackward0>)


Besides, to expand the capability and allow model to learn non-linear transformation between inputs and outputs, we will introduce the non-linear activation function between two linear layers. Here we use ReLU activation function:

![](https://github.com/Graph-and-Geometric-Learning/CPSC483-colab/blob/main/fig/relu.png?raw=1)

We can implement a ReLU functino by using `F.relu` or `nn.ReLU()`:

In [13]:
x = torch.rand(2, 3)
act_fn = nn.ReLU()

print("x:", x)
print("F.relu(x):", F.relu(x))
print("nn.relu(x):", act_fn(x))

x: tensor([[0.1524, 0.4639, 0.2016],
        [0.7785, 0.9239, 0.5848]])
F.relu(x): tensor([[0.1524, 0.4639, 0.2016],
        [0.7785, 0.9239, 0.5848]])
nn.relu(x): tensor([[0.1524, 0.4639, 0.2016],
        [0.7785, 0.9239, 0.5848]])


### Question 2 (10 points)

Now it is your time to build a MLP with linear layer and non-linear function. We use `nn.Module` to define a MLP class containing the basic modules of MLP. The modules we need will be defined in `__init__` function and the calculation will be performed in `forward` function.

In [14]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()

        # TODO: Define two Linear modules and a ReLU function.
        # The input size and output size of first Linear module should be input_dim and hidden_dim
        # The input size and output size of second Linear module should be hidden_dim and output_dim

        ############# Your code here ############
        ## (~3 line of code)
        self.lin1 = nn.Linear(input_dim, hidden_dim)
        self.lin2 = nn.Linear(hidden_dim, output_dim)
        self.act_relu = nn.ReLU()

        #########################################

    def forward(self, x):

        batch_size = x.shape[0]
        x = x.view(batch_size, -1)  # reshape the tensor to a 1-D vector

        out = 0

        # TODO: Use the modules you define in __init__ to perform calculation.
        # ReLU function should be used in the middle of two Linear modules.

        ############# Your code here ############
        ## (~3 line of code)
        hidden_out = self.lin1(x)
        relu_out = self.act_relu(hidden_out)
        out = self.lin2(relu_out)

        #########################################

        return out

As shown in the `MLP` class, we initialize the modules we need in `__init__` function, and perform the calculation of the model to predict the results in `forward` function. Here is an example to instantiate a MLP model:

In [15]:
model = MLP(input_dim=10, hidden_dim=20, output_dim=10)
print(model)  # show all the submodules

MLP(
  (lin1): Linear(in_features=10, out_features=20, bias=True)
  (lin2): Linear(in_features=20, out_features=10, bias=True)
  (act_relu): ReLU()
)


We can obtain the parameters of a module by its `parameters` functions, or `named_parameters` to get a name to each parameter object. For our MLP, we have the following parameters:

In [16]:
for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

Parameter lin1.weight, shape torch.Size([20, 10])
Parameter lin1.bias, shape torch.Size([20])
Parameter lin2.weight, shape torch.Size([10, 20])
Parameter lin2.bias, shape torch.Size([10])


## Datasets

We are going to use a dataset for image classification called MNIST. It is a large database of handwritten digits, and widely used for training and testing in the field of image processing. We can esily load this dataset with the help of PyTorch.


In [17]:
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

After importing required library, we can download the dataset of MNIST and save it in a folder. It will automatically create the folder if it doesn't exist.

In [18]:
ROOT = '.data'  # the folder for saving dataset

train_data = datasets.MNIST(root=ROOT,
                            train=True,
                            download=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 41.5MB/s]


Extracting .data/MNIST/raw/train-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.20MB/s]


Extracting .data/MNIST/raw/train-labels-idx1-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 10.3MB/s]


Extracting .data/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.23MB/s]

Extracting .data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/MNIST/raw



To improve the model's generalization and robustness, we usually will preprocess and augment our dataset. Data augmentation is a group of methods for creating new data points from existing data in order to increase the amount of data. These operations are included in `transforms.Compose` function. Here are some common transforms:

- `RandomRotation` - randomly rotates the image between `(-x, +x)` degrees, where we have set `x = 5`.
- `RandomCrop` - randomly taking a square crop of the image.
- `ToTensor()` - this converts the image from a PIL format into a PyTorch tensor.
- `Normalize` - this subtracts the mean and divides by the standard deviations given.


In [19]:
mean = train_data.data.float().mean() / 255
std = train_data.data.float().std() / 255

train_transforms = transforms.Compose([
                            transforms.RandomRotation(5, fill=(0,)),
                            transforms.RandomCrop(28, padding=2),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[mean], std=[std])
                                      ])

test_transforms = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[mean], std=[std])
                                     ])

Now with the transform function, we can load the train and test dataset of MNIST:

In [20]:
train_data = datasets.MNIST(root=ROOT,
                            train=True,
                            download=True,
                            transform=train_transforms)

test_data = datasets.MNIST(root=ROOT,
                           train=False,
                           download=True,
                           transform=test_transforms)



# Define the size of your training and validation datasets
train_size = 50000  # 50000 for training
val_size = len(train_data) - train_size  # 10000 for validation

# Split the dataset
train_data, val_data = data.random_split(train_data, [train_size, val_size])

print(f'Number of training examples: {len(train_data)}')  # check the number of the image in the train datasets
print(f'Number of training examples: {len(val_data)}')  # check the number of the image in the validation datasets
print(f'Number of testing examples: {len(test_data)}')  # check the number of the image in the test datasets

Number of training examples: 50000
Number of training examples: 10000
Number of testing examples: 10000


**NOTE**: The train/validation split is not ideal here because the validation dataset applies the `train_transform`, which is not what we want. However, we have kept it this way for simplicity.

In [21]:
print(train_data)

### Question 3 (5 points)

What is the label of the image with index 100 in the train dataset?

In [ ]:
def get_image_label(dataset, idx):
  # TODO: Implement a function that takes a dataset object,
  # an index of a image within the dataset, and returns the class/label
  # of the image (as an integer).

  label = -1

  ############# Your code here ############
  ## (~1 line of code)
  label =

  #########################################

  return label

idx = 100
label = get_image_label(train_data, idx)
print('Image with index {} has label {}'.format(idx, label))

### Question 4 (5 points)

What is the number of classes in the MNIST dataset?

In [ ]:
def get_num_classes(dataset):
  # TODO: Implement a function that takes a dataset object
  # and returns the number of classes for that dataset.

  num_classes = 0

  ############# Your code here ############
  ## (~1 line of code)

  #########################################

  return num_classes

num_classes = get_num_classes(train_data)
print("dataset has {} classes".format(num_classes))

We can have a look at these images with the help of library `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

image_idx = 50  # the index of the image we want to see
image = train_data[50][0]

fig = plt.figure()
plt.imshow(image.view(28, 28).cpu().numpy(), cmap='bone')  # plot the image
plt.axis('off')

`DataLoader` is also an important tool during our training. It can iterate over the dataset and yield a batch of images and labels to the model. The batchsize is a hyperparameter and we need to tune it during training. Here we set batchsize as 64, and define the dataloader for training and testing:

In [ ]:
BATCH_SIZE = 64

train_loader = data.DataLoader(train_data,
                                 shuffle=True,  # shuffle the dataset in every epoch
                                 batch_size=BATCH_SIZE)


val_loader = data.DataLoader(val_data,
                                batch_size=BATCH_SIZE)
test_loader = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

Now we need to initialize a MLP with appropriate `input_dim`, `hidden_dim` and `output_dim`. The image size is 28*28, and we will first flatten the image into a 784 element vector. So `input_dim` is 784 in MLP. `hidden_dim` is a tunable hyperparameter, and here we set it as 200. `output_dim` is the number of the classes. Please write the correct numbers and initialize a MLP:

In [ ]:
input_dim, hidden_dim, output_dim = 0, 0, 0

model = MLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

## Training and testing

Here is a total pipeline of training a model:

- pass a batch of data through the model and obtain the prediction
- compare the prediction with the label and calculate the loss of this batch
- calculate the gradient of each of the parameters with respect to the loss
- update the parameters with optimizer

To end-to-end train a model, we need to define a [optimizer](https://ruder.io/optimizing-gradient-descent/) and [loss function](https://towardsdatascience.com/common-loss-functions-in-machine-learning-46af0ffc4d23). Optimizer is a kind of algorithm used to updated the parameter effectively. Here we use Adam as our optimizer:

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

Then, we define loss function. How to choose a loss function? It depends on our current machine learning task:

- Mean Square Error: measured as the average of squared difference between predictions and actual observations. It is for regression task.
- Cross Entropy loss: computes the softmax activation function on the supplied predictions as well as the actual loss via negative log likelihood. It is for classification task.
- Mean Absolute Error: measured as the average of sum of absolute differences between predictions and actual observations. It is for regression task.

Our task is try to categorize the digit of the image, which is a kind of classification task. So here we define the Cross Entropy loss:

In [ ]:
loss_fn = nn.CrossEntropyLoss()

Now we can start to train our model! What we will do is:

- we put the model into `train` mode
- iterate over our dataloader, returning batches of (image, label)
- clear the gradients calculated from the last batch
- calculate the loss between our predictions and the actual labels
- calculate the gradients of each parameter
- update the parameters by taking an optimizer step
- update our metrics

In [ ]:
from tqdm import tqdm

def train(model, dataloader, optimizer, loss_fn):

    epoch_loss = 0
    model.train()

    for (x, y) in tqdm(dataloader, desc="Training"):  # iterate over our dataloader

        optimizer.zero_grad()  # clear the gradients

        y_pred = model(x)

        loss = loss_fn(y_pred, y)  # calculate the loss

        loss.backward()  # calculate the gradients of each parameter

        optimizer.step()  # update the parameters by taking an optimizer step

        # reweight the loss by the number of samples in the batch
        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

### Question 5 (5 points)

After training the model, we need to evaluate it. Here we first need to define a function to calculate the accuracy. It will compare the model prediction against the real class label:

In [ ]:
def calculate_accuracy(pred, label):
  # TODO: Implement the accuracy function. This function takes the
  # pred tensor and the label tensor. Take the index of the highest value for
  # your prediction and compares it against the actual class label.

  acc = 0.0

  ############# Your code here ############

  #########################################

  return acc

pred = torch.rand(5, 3)  # 5 examples, 3 classes
label = torch.tensor([0, 1, 2, 1, 0])  # the true label of each example
print("accuracy:", calculate_accuracy(pred, label))

### Question 6 (10 points)

Then we define the evaluation function, which is similar as training one:

- we put the model into `eval` mode
- we wrap the iterations inside a `with torch.no_grad()`
- iterate over our dataloader, returning batches of (image, label)
- calculate the loss between our predictions and the actual labels
- calculate the accuracy
- we do not calculate or zero gradients
- we do not take an optimizer step
- update our metrics

In [ ]:
def evaluate(model, dataloader, loss_fn):

    epoch_loss = 0
    epoch_acc = 0

    # TODO: Implement a evaluation function that takes the model,
    # dataloader, loss function and returns the average loss and accuracy.

    ############# Your code here ############
    ## (~8 lines of code)

    #########################################

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

The main difference between training process and evaluating process is:



Let's start to train our model!

In [ ]:
from time import time

EPOCHS = 5  # number of training epoch
CHECK_VAL_EVERY_N_EPOCH = 1

best_valid_loss = float('inf')

for epoch in range(EPOCHS):

    start_time = time()  # record the start time

    train_loss = train(model, train_loader, optimizer, loss_fn)

    end_time = time()

    epoch_time = end_time - start_time

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_time}s')
    print(f'\tTrain Loss: {train_loss:.3f}')

    if epoch % CHECK_VAL_EVERY_N_EPOCH == 0:
      val_loss, val_acc = evaluate(model, val_loader, loss_fn)
      print(f'Val Loss: {val_loss:.3f} | Val Acc: {val_acc*100:.2f}%')


torch.save(model.state_dict(), 'model.pt')  # saving model's parameters

After 5 epochs' training, we can load the parameter and evaluate the model:

In [ ]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_loader, loss_fn)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## Submission

Make sure to run all the cells and save a copy of this colab in your driver. If you complete this notebook, download the colab and upload your work to canvas to submit it.
